In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile

In [2]:
pars = AnalysisSchema.from_default().geometry_pars_obj

In [3]:
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)
pars.set('t_x', 1)
pars.set('t_y', 2)
pars.set('t_z', 3)

In [4]:
nrows = 100_000

fc = np.random.random(nrows) * 2048
sc = np.random.random(nrows) * 2048
om = np.random.random(nrows) * 360

In [5]:
cf = columnfile(new=True)
cf.nrows = nrows

cf.addcolumn(fc, 'fc')
cf.addcolumn(sc, 'sc')
cf.addcolumn(om, 'omega')

In [6]:
cf.parameters = pars

In [7]:
cf.updateGeometry()

In [8]:
cf.titles

['fc', 'sc', 'omega', 'xl', 'yl', 'zl', 'tth', 'eta', 'ds', 'gx', 'gy', 'gz']

In [9]:
gvecs = np.column_stack([cf.gx, cf.gy, cf.gz])

In [10]:
import ImageD11.transform, ImageD11.gv_general

In [11]:
import transform as mytrans

In [12]:
import importlib
importlib.reload(mytrans)

<module 'transform' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/transform.py'>

# Full pipeline test: sc, fc, omega, origins (zeroes for now) to g-vectors

In [13]:
gvecs_me = mytrans.det_to_g(cf.sc, cf.fc, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))),
                            pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                            pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'), jnp.array([1., 0, 0]))

In [14]:
gvecs_me

Array([[-0.25138913, -0.87785639, -1.20945785],
       [ 0.63495039,  0.36438958,  1.22634726],
       [ 0.03283245, -0.42351784,  1.0521766 ],
       ...,
       [ 1.29397681,  0.25920856, -1.02308987],
       [-0.93400725, -1.01701131,  0.61029224],
       [-1.41679064,  0.43506136,  1.52963969]], dtype=float64)

In [15]:
gvecs

array([[-0.25138913, -0.87785639, -1.20945785],
       [ 0.63495039,  0.36438958,  1.22634726],
       [ 0.03283245, -0.42351784,  1.0521766 ],
       ...,
       [ 1.29397681,  0.25920856, -1.02308987],
       [-0.93400725, -1.01701131,  0.61029224],
       [-1.41679064,  0.43506136,  1.52963969]])

In [16]:
assert np.allclose(gvecs_me, gvecs)

# Lab <-> Detector

In [17]:
%%time

xyz_id11 = ImageD11.transform.compute_xyz_lab((cf.sc, cf.fc), **pars.parameters)

CPU times: user 46.2 ms, sys: 3.02 ms, total: 49.2 ms
Wall time: 1.63 ms


In [18]:
%%time

xyz_me = mytrans.det_to_xyz_lab(cf.sc, cf.fc, pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

CPU times: user 2.58 s, sys: 261 ms, total: 2.84 s
Wall time: 60.6 ms


In [19]:
assert np.allclose(xyz_id11.T, xyz_me)

In [20]:
v_det_me = mytrans.xyz_lab_to_det(cf.xl, cf.yl, cf.zl,
                                  pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [21]:
assert np.allclose(v_det_me[:, 0], cf.sc)
assert np.allclose(v_det_me[:, 1], cf.fc)

# Lab <-> tth, eta, omega

In [22]:
tth_id11, eta_id11 = ImageD11.transform.compute_tth_eta_from_xyz(np.stack((cf.xl, cf.yl, cf.zl)), cf.omega, **pars.parameters)

In [23]:
tth_me, eta_me = mytrans.xyz_lab_to_tth_eta(jnp.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'))

In [24]:
assert np.allclose(tth_me, tth_id11)
assert np.allclose(eta_me, eta_id11)

In [25]:
fc_id11, sc_id11 = ImageD11.transform.compute_xyz_from_tth_eta(cf.tth, cf.eta, cf.omega, **pars.parameters)

In [26]:
assert np.allclose(fc_id11, cf.fc)
assert np.allclose(sc_id11, cf.sc)

In [27]:
sc_me, fc_me = mytrans.tth_eta_omega_to_det(cf.tth, cf.eta, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))),
                                          pars.get('wedge'), pars.get('chi'),  pars.get('wavelength'),
                                          pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                          pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                          pars.get('tilt_x'),
                                          pars.get('distance'),
                                          pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'),  jnp.array([1., 0, 0]))

In [28]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

In [29]:

dxyzl = mytrans.det_to_xyz_lab(sc, fc, pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [30]:
dxyzl.shape

(100000, 3)

In [31]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [32]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [33]:
assert np.allclose(t_id11, t_me.T)

In [34]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 449 ms, sys: 38.2 ms, total: 487 ms
Wall time: 32.5 ms


In [35]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me), valid = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), jnp.array([1., 0, 0]) )

CPU times: user 8.63 s, sys: 798 ms, total: 9.42 s
Wall time: 226 ms


In [36]:
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [37]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [38]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [39]:
t_id11.shape

(3, 100000)

In [40]:
t_me.shape

(100000, 3)

In [41]:
assert np.allclose(t_id11, t_me.T)

In [42]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 425 ms, sys: 39.8 ms, total: 465 ms
Wall time: 32.5 ms


In [43]:
tth_id11

array([24.90467812, 23.45033525, 18.58208828, ..., 27.48866183,
       24.8082135 , 35.28175083])

In [44]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me), valid = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), jnp.array([1., 0, 0]))

CPU times: user 15.5 ms, sys: 1.86 ms, total: 17.4 ms
Wall time: 373 μs


In [45]:
tth_me

Array([24.90467812, 23.45033525, 18.58208828, ..., 27.48866183,
       24.8082135 , 35.28175083], dtype=float64)

In [46]:
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [47]:
eta1_me

Array([144.79134287,  28.25108249,  19.29334013, ..., 129.23044707,
        65.78695194,  40.52888501], dtype=float64)

In [48]:
eta1_id11

array([144.79134287,  28.25108249,  19.29334013, ..., 129.23044707,
        65.78695194,  40.52888501])

In [49]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))

In [50]:
k_id11

array([[-0.32677455, -0.29024322, -0.18319517, ..., -0.39673068,
        -0.32428763, -0.64545855],
       [-0.85319249,  0.61847684, -0.36998841, ...,  1.2914969 ,
         1.32277488,  1.26425373],
       [-1.20908957,  1.25423867,  1.05691487, ..., -0.98128409,
         0.65133886,  1.58789718]])

In [51]:
k_me = mytrans.tth_eta_to_k(cf.tth, cf.eta, pars.get('wavelength'))

In [52]:
k_me

Array([[-0.32677455, -0.85319249, -1.20908957],
       [-0.29024322,  0.61847684,  1.25423867],
       [-0.18319517, -0.36998841,  1.05691487],
       ...,
       [-0.39673068,  1.2914969 , -0.98128409],
       [-0.32428763,  1.32277488,  0.65133886],
       [-0.64545855,  1.26425373,  1.58789718]], dtype=float64)

In [53]:
k_id11.shape

(3, 100000)

In [54]:
k_me.shape

(100000, 3)

In [55]:
assert np.allclose(k_id11.T, k_me)

In [56]:
# test computation of angles for g-vectors

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))
pre = (C @ W).T

oms_id11 = ImageD11.gv_general.g_to_k(gvecs.T, pars.get('wavelength'), axis=[0,0,1], pre=pre, post=None)

In [57]:
oms_id11

(array([138.82395794, -85.75205875, 167.19638888, ..., -95.60629557,
        125.00825889,  45.43616786]),
 array([  0.78051167, 138.04435216,  35.19784863, ..., 120.81480611,
        -27.5473897 , -79.46204672]),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [58]:
oms_me = mytrans.omega_solns_for_g(gvecs, pars.get('wavelength'), np.array([0,0,1]), pre, jnp.eye(3), jnp.array([1., 0, 0]))

In [59]:
oms_me

(Array([138.82395794, -85.75205875, 167.19638888, ..., -95.60629557,
        125.00825889,  45.43616786], dtype=float64),
 Array([  0.78051167, 138.04435216,  35.19784863, ..., 120.81480611,
        -27.5473897 , -79.46204672], dtype=float64),
 Array([ True,  True,  True, ...,  True,  True,  True], dtype=bool))

In [60]:
assert np.allclose(oms_id11[0], oms_me[0])
assert np.allclose(oms_id11[1], oms_me[1])
assert np.allclose(oms_id11[2], oms_me[2])

In [61]:
# now use k vectors to test g_from_k

In [62]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
g_id11 = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [63]:
g_id11

array([[-0.25138913,  0.63495039,  0.03283245, ...,  1.29397681,
        -0.93400725, -1.41679064],
       [-0.87785639,  0.36438958, -0.42351784, ...,  0.25920856,
        -1.01701131,  0.43506136],
       [-1.20945785,  1.22634726,  1.0521766 , ..., -1.02308987,
         0.61029224,  1.52963969]])

In [64]:
g_me = mytrans.lab_to_sample(k_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [65]:
g_me

Array([[-0.25138913, -0.87785639, -1.20945785],
       [ 0.63495039,  0.36438958,  1.22634726],
       [ 0.03283245, -0.42351784,  1.0521766 ],
       ...,
       [ 1.29397681,  0.25920856, -1.02308987],
       [-0.93400725, -1.01701131,  0.61029224],
       [-1.41679064,  0.43506136,  1.52963969]], dtype=float64)

In [66]:
assert np.allclose(g_id11.T, g_me)

In [67]:
g_id11_trans = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [68]:
g_id11_trans

array([[-0.25138913,  0.63495039,  0.03283245, ...,  1.29397681,
        -0.93400725, -1.41679064],
       [-0.87785639,  0.36438958, -0.42351784, ...,  0.25920856,
        -1.01701131,  0.43506136],
       [-1.20945785,  1.22634726,  1.0521766 , ..., -1.02308987,
         0.61029224,  1.52963969]])

In [69]:
# C @ W -Z works

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))

post = C @ W

g_id11_gvgeneral = ImageD11.gv_general.k_to_g(k_id11, cf.omega, axis=np.array([0., 0., -1]), pre=None, post=post)

In [70]:
g_id11_gvgeneral

array([[-0.25138913,  0.63495039,  0.03283245, ...,  1.29397681,
        -0.93400725, -1.41679064],
       [-0.87785639,  0.36438958, -0.42351784, ...,  0.25920856,
        -1.01701131,  0.43506136],
       [-1.20945785,  1.22634726,  1.0521766 , ..., -1.02308987,
         0.61029224,  1.52963969]])

In [71]:
assert np.allclose(g_id11_trans, g_id11_gvgeneral)

In [72]:
g_id11 = ImageD11.transform.compute_g_vectors(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [73]:
g_id11

array([[-0.25138913,  0.63495039,  0.03283245, ...,  1.29397681,
        -0.93400725, -1.41679064],
       [-0.87785639,  0.36438958, -0.42351784, ...,  0.25920856,
        -1.01701131,  0.43506136],
       [-1.20945785,  1.22634726,  1.0521766 , ..., -1.02308987,
         0.61029224,  1.52963969]])

In [74]:
assert np.allclose(gvecs, g_id11.T)

In [75]:
g_me = mytrans.tth_eta_omega_to_g(cf.tth, cf.eta, cf.omega,  pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),)

In [76]:
g_me

Array([[-0.25138913, -0.87785639, -1.20945785],
       [ 0.63495039,  0.36438958,  1.22634726],
       [ 0.03283245, -0.42351784,  1.0521766 ],
       ...,
       [ 1.29397681,  0.25920856, -1.02308987],
       [-0.93400725, -1.01701131,  0.61029224],
       [-1.41679064,  0.43506136,  1.52963969]], dtype=float64)

In [77]:
assert np.allclose(gvecs, g_me)

In [78]:
assert np.allclose(gvecs, mytrans.sample_to_lab(mytrans.lab_to_sample(gvecs, cf.omega, pars.get('wedge'), pars.get('chi')), cf.omega, pars.get('wedge'), pars.get('chi')))

In [79]:
# g-vectors to (tth, eta, omega)
tth, [eta_one, eta_two], [omega1, omega2], valid = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), jnp.array([1., 0, 0]))

In [80]:
assert np.allclose(tth, cf.tth)
assert all(np.logical_or(np.isclose(cf.eta, eta_one), np.isclose(cf.eta, eta_two)))
assert all(np.logical_or(np.isclose(cf.omega, omega1 % 360), np.isclose(cf.omega, omega2 % 360)))

In [81]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_g(tth, eta_one, omega1, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [82]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_g(tth, eta_two, omega2, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [83]:
k_me = mytrans.xyz_lab_to_k(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), jnp.array([1., 0, 0]))

In [84]:
k_me

Array([[-0.32677455, -0.85319249, -1.20908957],
       [-0.29024322,  0.61847684,  1.25423867],
       [-0.18319517, -0.36998841,  1.05691487],
       ...,
       [-0.39673068,  1.2914969 , -0.98128409],
       [-0.32428763,  1.32277488,  0.65133886],
       [-0.64545855,  1.26425373,  1.58789718]], dtype=float64)

In [85]:
sc_me, fc_me = mytrans.k_to_det(k_me, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'), jnp.array([1., 0, 0]))

In [86]:
sc_me

Array([1903.62381687,  337.28926644,  475.31987817, ..., 1759.97354386,
        707.05680811,   19.58182572], dtype=float64)

In [87]:
xyz_me = mytrans.k_to_xyz_lab(k_me, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'), jnp.array([1., 0, 0]))

In [88]:
assert np.allclose(xyz_me, np.column_stack((cf.xl, cf.yl, cf.zl)))

In [89]:
xyz_me

Array([[155607.06452712, -41651.50428097, -59025.5992387 ],
       [150143.5974234 ,  28805.71699693,  58417.62586267],
       [151412.97993849, -16817.59343415,  48046.85259325],
       ...,
       [153134.23296899,  63444.31030564, -48201.48438728],
       [150429.69926972,  62379.39305878,  30719.69679582],
       [148587.85329191,  65485.3409835 ,  82251.31110325]],      dtype=float64)